In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from functools import partial

import optuna
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

In [2]:
df = pd.read_csv('reference/major_results_2020.csv')

In [3]:
df.head(4)

,都道府県名,都道府県・市区町村名,都道府県・市区町村名（英語）,市などの別,総数（人）,男（人）,女（人）,2015年（平成27年）の人口（組替）（人）,5年間の人口増減数（人）,5年間の人口増減率（％）,...,一般世帯（世帯）.1,核家族世帯（世帯）,夫婦のみの世帯（世帯）,夫婦と子供から成る世帯（世帯）,男親と子供から成る世帯（世帯）,女親と子供から成る世帯（世帯）,単独世帯（世帯）,65歳以上の単独世帯（世帯）,夫65歳以上，妻60歳以上の夫婦のみの世帯（世帯）,3世代世帯（世帯）
0,00_全国,00000_全国,Japan,a,126146099,61349581,64796518,127094745,-948646,-0.74641,...,55704949,30110571,11158840,13949190,738006,4264535,21151042,6716806,6533895,2337703
1,01_北海道,01000_北海道,Hokkaido,a,5224614,2465088,2759526,5381733,-157119,-2.91949,...,2469063,1324406,584819,511571,29921,198095,999825,361735,345741,59601
2,01_北海道,01100_札幌市,Sapporo-shi,1,1973395,918682,1054713,1952356,21039,1.07762,...,967372,496691,204909,204131,10175,77476,422160,121789,110890,15727
3,01_北海道,01101_札幌市中央区,Sapporo-shi Chuo-ku,0,248680,112853,135827,237627,11053,4.65141,...,141223,54941,24467,20812,878,8784,80425,16447,11059,1111


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1965 entries, 0 to 1964
Data columns (total 49 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   都道府県名                        1965 non-null   object 
 1   都道府県・市区町村名                   1965 non-null   object 
 2   都道府県・市区町村名（英語）               1965 non-null   object 
 3   市などの別                        1965 non-null   object 
 4   総数（人）                        1965 non-null   object 
 5   男（人）                         1965 non-null   object 
 6   女（人）                         1965 non-null   object 
 7   2015年（平成27年）の人口（組替）（人）       1965 non-null   object 
 8   5年間の人口増減数（人）                 1965 non-null   int64  
 9   5年間の人口増減率（％）                 1965 non-null   object 
 10  面積（参考）（km2）                  1965 non-null   float64
 11  人口密度（人/km2）                  1965 non-null   float64
 12  平均年齢（歳）                      1965 non-null   object 
 13  年齢中位数（歳）          

In [5]:
df.columns

Index(['都道府県名', '都道府県・市区町村名', '都道府県・市区町村名（英語）', '市などの別', '総数（人）', '男（人）',
       '女（人）', '2015年（平成27年）の人口（組替）（人）', '5年間の人口増減数（人）', '5年間の人口増減率（％）',
       '面積（参考）（km2）', '人口密度（人/km2）', '平均年齢（歳）', '年齢中位数（歳）',
       '年齢別人口（男女「総数」）15歳未満（人）', '年齢別人口（男女「総数」）15～64歳（人）',
       '年齢別人口（男女「総数」）65歳以上（人）', '人口構成比［年齢別］(男女「総数」）15歳未満（％）',
       '人口構成比［年齢別］(男女「総数」）15～64歳（％）', '人口構成比［年齢別］(男女「総数」）65歳以上（％）',
       '年齢別人口（男）15歳未満（人）', '年齢別人口（男）15～64歳（人）', '年齢別人口（男）65歳以上（人）',
       '人口構成比［年齢別］(男）15歳未満（％）', '人口構成比［年齢別］(男）15～64歳（％）',
       '人口構成比［年齢別］(男）65歳以上（％）', '年齢別人口（女）15歳未満（人）', '年齢別人口（女）15～64歳（人）',
       '年齢別人口（女）65歳以上（人）', '人口構成比［年齢別］(女）15歳未満（％）', '人口構成比［年齢別］(女）15～64歳（％）',
       '人口構成比［年齢別］(女）65歳以上（％）', '人口性比（％）', '日本人（人）', '外国人（人）', '総世帯（世帯）',
       '一般世帯（世帯）', '施設等の世帯（世帯）', '2015年（平成27年）の世帯数（組替）（世帯）', '一般世帯（世帯）.1',
       '核家族世帯（世帯）', '夫婦のみの世帯（世帯）', '夫婦と子供から成る世帯（世帯）', '男親と子供から成る世帯（世帯）',
       '女親と子供から成る世帯（世帯）', '単独世帯（世帯）', '65歳以上の単独世帯（世帯）',
       '夫65歳以上，妻60歳以上の夫婦のみの世帯（世帯）', '3世代世帯（世帯）'],


In [6]:
dropcol = ['都道府県名', '都道府県・市区町村名（英語）', '一般世帯（世帯）.1', '2015年（平成27年）の人口（組替）（人）', '5年間の人口増減数（人）', '5年間の人口増減率（％）', '2015年（平成27年）の世帯数（組替）（世帯）']
df = df.drop(dropcol, axis=1)

In [7]:
df['市区町村コード'] = df['都道府県・市区町村名'].str.split('_',expand=True)[0].astype(int).astype(object)
df['都道府県・市区町村名'] = df['都道府県・市区町村名'].str.split('_',expand=True)[1]


In [8]:
# 区および、市（政令市を除く）、町、村のレコードを取得
# 政令市のデータ、都道府県のデータも取得する。
sikutyouson = df[(df['市などの別'] == '0') | (df['市などの別'] == '2')
                 | (df['市などの別'] == '3')]
seireishi = df[df['市などの別'] == '1']
todouhuken = df[df['市などの別'] == 'a']
sikutyouson = sikutyouson.set_index(['市区町村コード'])

/usr/local/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [9]:
sikutyouson.isnull().sum()

都道府県・市区町村名                     0
市などの別                          0
総数（人）                          0
男（人）                           0
女（人）                           0
面積（参考）（km2）                    0
人口密度（人/km2）                    0
平均年齢（歳）                        0
年齢中位数（歳）                       0
年齢別人口（男女「総数」）15歳未満（人）          0
年齢別人口（男女「総数」）15～64歳（人）         0
年齢別人口（男女「総数」）65歳以上（人）          0
人口構成比［年齢別］(男女「総数」）15歳未満（％）     0
人口構成比［年齢別］(男女「総数」）15～64歳（％）    0
人口構成比［年齢別］(男女「総数」）65歳以上（％）     0
年齢別人口（男）15歳未満（人）               0
年齢別人口（男）15～64歳（人）              0
年齢別人口（男）65歳以上（人）               0
人口構成比［年齢別］(男）15歳未満（％）          0
人口構成比［年齢別］(男）15～64歳（％）         0
人口構成比［年齢別］(男）65歳以上（％）          0
年齢別人口（女）15歳未満（人）               0
年齢別人口（女）15～64歳（人）              0
年齢別人口（女）65歳以上（人）               0
人口構成比［年齢別］(女）15歳未満（％）          0
人口構成比［年齢別］(女）15～64歳（％）         0
人口構成比［年齢別］(女）65歳以上（％）          0
人口性比（％）                        0
日本人（人）                         0
外国人（人）                         0
総世帯（世帯）   

In [10]:
sikutyouson = sikutyouson.drop(['市などの別'], axis = 1)

In [11]:
for colname in sikutyouson.columns:
    if colname == '都道府県・市区町村名':
        continue
    sikutyouson[sikutyouson[colname] == '-'] = 0
    sikutyouson[colname] = sikutyouson[colname].astype(float)

In [12]:
sikutyouson.to_csv('reference/population.csv', index=True, header=True)